In [31]:
import os
import pickle
import click
import mlflow

from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error


In [32]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")


In [34]:
HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models0"
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state']

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(EXPERIMENT_NAME)
#mlflow.sklearn.autolog(log_datasets=False)



2024/05/29 20:00:15 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models0' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///C:/Users/SebastianSy/Desktop/231110_Backup Gateway '
 'Final/Administration/Other/Sinn/Start-up and Job/Programmieren/MLOps '
 'Zoomcamp/week 2/mlruns/6'), creation_time=1717005615455, experiment_id='6', last_update_time=1717005615455, lifecycle_stage='active', name='random-forest-best-models0', tags={}>

In [35]:
def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)




In [36]:
def train_and_log_model(data_path, params):
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    with mlflow.start_run():
        new_params = {}
        for param in RF_PARAMS:
            new_params[param] = int(params[param])

        rf = RandomForestRegressor(**new_params)
        rf.fit(X_train, y_train)

        # Evaluate model on the validation and test sets
        val_rmse = root_mean_squared_error(y_val, rf.predict(X_val))
        mlflow.log_metric("val_rmse", val_rmse)
        test_rmse = root_mean_squared_error(y_test, rf.predict(X_test))
        mlflow.log_metric("test_rmse", test_rmse)
        mlflow.sklearn.log_model(RandomForestRegressor, artifact_path="artifact")
        mlflow.set_tag("model", rf)
        mlflow.log_param("max_depth", new_params["max_depth"])
        mlflow.log_param("min_samples_leaf", new_params["min_samples_leaf"])
        mlflow.log_param("min_samples_split", new_params["min_samples_split"])
        mlflow.log_param("n_estimators", new_params["n_estimators"])
        mlflow.log_param("random_state", new_params["random_state"])

        mlflow.end_run()


In [37]:
def run_register_model(data_path: str, top_n: int):

    client = MlflowClient()

    # Retrieve the top_n model runs and log the models
    experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=top_n,
        order_by=["metrics.rmse ASC"]
    )
    for run in runs:
        train_and_log_model(data_path=data_path, params=run.data.params)

    # Select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    # best_run = client.search_runs( ...  )[0]

    # Register the best model
    # mlflow.register_model( ... )



In [38]:
run_register_model(data_path="./output", top_n=5)